# Algorithm for higher order derivatives of the Weierstrass sigma function

In [1]:
from sympy import *
from sympy.core.relational import Equality

(z, g2, g3, n, m) = symbols('z, g2, g3, n, m')
pw = Function('pw') # Weierstrass P function
pwp = Function('pwp') # Derivative of Weierstrass P function
zw = Function('zw') # Weierstrass zeta function
sigma = Function('sigma') # Weierstrass sigma function
from wpk import wpk, wsk, run_tests

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%load_ext autoreload
%autoreload 2

In [4]:
run_tests()

True
True
True
True
True
True
True
True
True


In [ ]:
Eq(Derivative(zw(z, g2, g3), z), -pw(z, g2, g3))

In [ ]:
def nth_diff_zeta(nth_order: int) -> Equality:
    
    """
    Builds the symbolic equation for the nth_order derivative of the Weierstrass zeta function.
    Odd orders: result is an O((n-1)/2 + 1) polynomial in the Weierstrass P function.
    Even orders: result is an O(n/2 - 1) polynomial in the Weierstrass P function, multiplied by P Prime.
    
    Args: nth_order (int > 0)
    
    Returns: (sympy.core.relational.Equality)
    
    Params: g2, g3, Weierstrass elliptic invariants
    """
    if nth_order < 1:
        print("nth_order must not be less than 1")
        raise
        
    dzw_pw = Eq(diff(zw(z, g2, g3), (z,nth_order)), -diff(pw(z, g2, g3),(z,nth_order-1)))
    if nth_order == 1:
        return dzw_pw
    nth_order_diff_eq = dzw_pw.subs(*nth_diff_pw(nth_order-1).args)
        
    return nth_order_diff_eq

In [ ]:
def nth_diff_sigma(nth_order: int) -> Equality:
    
    """
    Builds the symbolic equation for the nth_order derivative of the Weierstrass sigma function.
    The result is a polynomial in the Weierstrass P, P Prime, zeta, and sigma functions.
    
    Args: nth_order (int > 0)
    
    Returns: (sympy.core.relational.Equality)
    
    Params: g2, g3, Weierstrass elliptic invariants
    """
    
    if nth_order < 1:
        print("nth_order must not be less than 1")
        raise

        
    def _nth_diff_sigma_nth_diff_zw(nth_order):
        if nth_order < 1:
            print("nth_order must not be less than 1")
            raise
        return Eq(diff(sigma(z, g2, g3), (z,nth_order)), 
                  diff(sigma(z, g2, g3)*zw(z, g2, g3), (z,nth_order-1)).doit()
                 )
    
    if nth_order == 1:
        return _nth_diff_sigma_nth_diff_zw(nth_order)
        
    # Calculate all order diff sigma in terms of diff zw
    all_orders_dsigma_dzw = [_nth_diff_sigma_nth_diff_zw(_n) for _n in range(1, nth_order + 1)]
    nth_order_dsigma_dzw = all_orders_dsigma_dzw[-1]
    for nth_dsigma_dzw in all_orders_dsigma_dzw[0:-1][::-1]:
        nth_order_dsigma_dzw = Eq(nth_order_dsigma_dzw.lhs,
                                  nth_order_dsigma_dzw.rhs.subs(*nth_dsigma_dzw.args))
        
    # Substitute for derivatives of Weierstrass zeta in terms of polynomials in Weierstrass P and P Prime
    all_orders_dzw_dpw_args = [nth_diff_zeta(_n).args for _n in range(1, nth_order)][::-1]
    nth_order_dsigma_dzw = nth_order_dsigma_dzw.subs(all_orders_dzw_dpw_args).expand()
        
    return nth_order_dsigma_dzw

In [ ]:
nth_diff_sigma(1).args
nth_diff_sigma(2).args
nth_diff_sigma(3).args

In [ ]:
ch3 = Eq(Derivative(sigma(z, g2, g3), (z, 3)),
 -pw(z, g2, g3)*sigma(z, g2, g3)*zw(z, g2, g3) + sigma(z, g2, g3)*zw(z, g2, g3)**3 + 2*sigma(z, g2, g3)*zw(z, g2, g3)*Derivative(zw(z, g2, g3), z) - sigma(z, g2, g3)*Derivative(pw(z, g2, g3), z))

In [ ]:
nth_diff_sigma(3).lhs - ch3.lhs
nth_diff_sigma(3).rhs - ch3.rhs.subs([(diff(pw(z,g2,g3),z), pwp(z,g2,g3)), (diff(zw(z, g2, g3), z), -pw(z, g2, g3))])